# Implementation of the Kuribayashi BERT minus model

## libraries

In [1]:
!pip install transformers --upgrade
!pip install ipywidgets
!pip install IProgress
!pip install datasets
!pip install torch-lr-finder

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import transformers
from transformers import BertTokenizer, BertConfig
from transformers import BertModel, BertForSequenceClassification
from transformers import BatchEncoding, default_data_collator, DataCollatorWithPadding

import torch
import torch.nn as nn

import numpy as np

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

import datasets
from datasets import load_metric

from torch.utils.data import DataLoader

from tqdm import tqdm
from operator import itemgetter

In [3]:
print(transformers.__version__)

4.25.1


## tokenizer

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

## data

In [6]:
DATA_FILE = '/notebooks/KURI-BERT/notebooks/full_formula_w_fts/pe_dataset_for_bert_minus_w_fts_combined_corrected.pt'

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Load data

In [8]:
dataset = torch.load(DATA_FILE)

### preprocessing

In [11]:
MAX_LENGTH = 0
MAX_SPAN = 0

for split in ['train', 'test', 'validation']:
    
    for col_name in ['fts_spans']:
        
        for x in dataset[split][col_name]:
            
            
            if max(x,key=itemgetter(1))[1] > MAX_SPAN:
                
                MAX_SPAN = max(x,key=itemgetter(1))[1]
                MAX_SPAN = min(MAX_SPAN, tokenizer.model_max_length - 2)
            
            if len(x) > MAX_LENGTH:
                
                MAX_LENGTH = len(x)

In [12]:
def get_padding(batch, padding_target):    
    
    if padding_target == 'am_spans':
        
        col_name = 'am_spans'
        padding_val = [[-1,-1]]
        max_length = MAX_LENGTH
        
    elif padding_target == 'ac_spans':
        
        col_name = 'ac_spans'
        padding_val = [[-1,-1]]
        max_length = MAX_LENGTH
        
    elif padding_target == 'fts_spans':
        
        col_name = 'fts_spans'
        padding_val = [[-1,-1]]
        max_length = MAX_LENGTH
        
    elif padding_target == 'label':
    
        col_name = 'paragraph_labels_numeric'
        padding_val = [-100]      
        max_length = MAX_LENGTH

    padded_spans = []

    for idx, span in enumerate(batch[col_name]):

        padded_span = batch[col_name][idx] + (max_length - len(span)) * padding_val
        padded_spans.append(padded_span)

    return padded_spans         

In [13]:
def get_combined_spans(am_spans_ll, ac_spans_ll, fts_spans_ll):
    
    spans_ll = []
    
    for am_spans, ac_spans, fts_spans in zip(am_spans_ll, ac_spans_ll, fts_spans_ll):
        
        spans = []
        
        for am_span, ac_span, fts_span in zip(am_spans, ac_spans, fts_spans):
            
            for idx in [0,1]:
                
                if am_span[idx] > MAX_SPAN:
                    am_span[idx] = MAX_SPAN
                if ac_span[idx] > MAX_SPAN:
                    ac_span[idx] = MAX_SPAN
                if fts_span[idx] > MAX_SPAN:
                    fts_span[idx] = MAX_SPAN

            span = [am_span, ac_span, fts_span]
            spans.extend(span)
            
        spans_ll.append(spans)

    return spans_ll

### tokenize 

In [14]:
def tokenize(batch):
    
    tokenized_text = tokenizer(batch['full_paragraph'], truncation=True, padding=True, max_length=512)
    tokenized_text['label'] = get_padding(batch, 'label')
    tokenized_text['am_spans'] = get_padding(batch, 'am_spans')
    tokenized_text['ac_spans'] = get_padding(batch, 'ac_spans')
    tokenized_text['fts_spans'] = get_padding(batch, 'fts_spans')
    tokenized_text['spans'] = get_combined_spans(tokenized_text['am_spans'], tokenized_text['ac_spans'], tokenized_text['fts_spans'])      
    
    return tokenized_text

In [15]:
dataset = dataset.map(tokenize, batched=True, batch_size=len(dataset['train']))

Parameter 'function'=<function tokenize at 0x7ff65133b280> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
dataset['test'].features['spans'] = datasets.Array2D(shape=(36, 2), dtype="int32")
dataset['train'].features['spans'] = datasets.Array2D(shape=(36, 2), dtype="int32")
dataset['validation'].features['spans'] = datasets.Array2D(shape=(36, 2), dtype="int32")

In [18]:
dataset = dataset.map(lambda batch: batch, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'spans', 'label'])

## span representation function

In [20]:
def minus_one(t):
    
    return torch.where(t == 0, 0, t-1)

def plus_one(t):
    
    return torch.where(t >= MAX_SPAN, MAX_SPAN, t+1)

In [21]:
def get_span_representations(outputs, spans):

    
    batch_size = spans.shape[0]
    
    nr_span_indices = spans.shape[1]
    
    
    idx_l_ams = range(0, nr_span_indices, 3)
    idx_l_acs = range(1, nr_span_indices, 3)
    idx_l_fts = range(2, nr_span_indices, 3)
    
    # Add 1 to all span indices (both am and ac) to offset for the CLS token in the input_ids.
    
    am_spans = spans[:, idx_l_ams, :] + 1 
    ac_spans = spans[:, idx_l_acs, :] + 1
    fts_spans = spans[:, idx_l_fts, :] + 1
    
    
    am_spans_minus_one = minus_one(am_spans)
    am_spans_plus_one = plus_one(am_spans)
    
    ac_spans_minus_one = minus_one(ac_spans)
    ac_spans_plus_one = plus_one(ac_spans)
    
    fts_spans_minus_one = minus_one(fts_spans)
    fts_spans_plus_one = plus_one(fts_spans)
    
    
    am_spans = am_spans.flatten(start_dim=1)
    ac_spans = ac_spans.flatten(start_dim=1)
    fts_spans = fts_spans.flatten(start_dim=1)
    
    nr_adus = ac_spans.shape[1] // 2
    
    
    am_spans_minus_one = am_spans_minus_one.flatten(start_dim=1)
    am_spans_plus_one = am_spans_plus_one.flatten(start_dim=1)
    
    ac_spans_minus_one = ac_spans_minus_one.flatten(start_dim=1)
    ac_spans_plus_one = ac_spans_plus_one.flatten(start_dim=1)
    
    fts_spans_minus_one = fts_spans_minus_one.flatten(start_dim=1)
    fts_spans_plus_one = fts_spans_plus_one.flatten(start_dim=1)
    
    ############# FOR AMs #################
    
    outputs_am = outputs[:,am_spans,:]
    outputs_am = torch.cat([outputs_am[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_am = outputs_am.reshape(batch_size, nr_adus * 2, -1)
    
    
    outputs_am_minus_one = outputs[:,am_spans_minus_one,:]
    outputs_am_minus_one = torch.cat([outputs_am_minus_one[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_am_minus_one = outputs_am_minus_one.reshape(batch_size, nr_adus * 2, -1)
    
    outputs_am_plus_one = outputs[:,am_spans_plus_one,:]
    outputs_am_plus_one = torch.cat([outputs_am_plus_one[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_am_plus_one = outputs_am_plus_one.reshape(batch_size, nr_adus * 2, -1)
    
    
    
    ### Now that we have outputs_am i.e. outputs at am_span indices, now create the four Kuri forumlas for AMs
    
    # ============== the corrected 1st one =================== 
    
    outputs_am_first_term = torch.cat([outputs_am[:,i+1,:] - outputs_am_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1) # i + 1 here means j in kuri, i here means i in kuri
    outputs_am_first_term = outputs_am_first_term.reshape(batch_size, -1, 768)
    
     # ============== the corrected 2nd one ==================
    
    outputs_am_second_term = torch.cat([outputs_am[:,i,:] - outputs_am_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1) # changed + 2 to + 1 to make it run # changed from +1 to +2 to ensure +2 is not a problem for AMs
    outputs_am_second_term = outputs_am_second_term.reshape(batch_size, -1, 768)
    
    
        # ============== the corrected third one ================== 
    
    outputs_am_third_term = torch.cat([outputs_am_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_am_third_term = outputs_am_third_term.reshape(batch_size, -1, 768)
    

        # ============== the corrected fourth one ==================
    
    outputs_am_fourth_term = torch.cat([outputs_am_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1) # changed + 2 to + 1 to make it run # changed from +1 to +2 to ensure +2 is not a problem for AMs
    outputs_am_fourth_term = outputs_am_fourth_term.reshape(batch_size, -1, 768)
    
    # ============== NOW CONCATENATE THEM =========
    
    
    am_minus_representations = torch.cat([outputs_am_first_term, outputs_am_second_term, outputs_am_third_term, outputs_am_fourth_term], dim=-1)   
    
    
    
    ### am minus span representation according to kuribayashi paper is now above.
    
    
    outputs_ac = outputs[:,ac_spans,:]
    outputs_ac = torch.cat([outputs_ac[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_ac = outputs_ac.reshape(batch_size, nr_adus * 2, -1)
    
    
    outputs_ac_minus_one = outputs[:,ac_spans_minus_one,:] # xxx. added to solve bug 2
    outputs_ac_minus_one = torch.cat([outputs_ac_minus_one[i,i,:,:] for i in range(batch_size)], dim=0) # xxx. added to solve bug 2
    outputs_ac_minus_one = outputs_ac_minus_one.reshape(batch_size, nr_adus * 2, -1) # xxx. added to solve bug 2
    
    outputs_ac_plus_one = outputs[:,ac_spans_plus_one,:] # xxx. added to solve bug 2
    outputs_ac_plus_one = torch.cat([outputs_ac_plus_one[i,i,:,:] for i in range(batch_size)], dim=0) # xxx. added to solve bug 2
    outputs_ac_plus_one = outputs_ac_plus_one.reshape(batch_size, nr_adus * 2, -1) # xxx. added to solve bug 2
    
    ### Now that we have outputs_ac i.e. outputs at ac_span indices, now create the four Kuri forumlas for ACs
    
    
    # ============== the corrected first one ===================
    
    outputs_ac_first_term = torch.cat([outputs_ac[:,i+1,:] - outputs_ac_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_ac_first_term = outputs_ac_first_term.reshape(batch_size, -1, 768)
    
        # ============== the correct second one ==================
    
    outputs_ac_second_term = torch.cat([outputs_ac[:,i,:] - outputs_ac_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1) # changed + 2 to + 1 to make it run
    outputs_ac_second_term = outputs_ac_second_term.reshape(batch_size, -1, 768)
    
    
    # ============== the corrected third term ==================
    
    outputs_ac_third_term = torch.cat([outputs_ac_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_ac_third_term = outputs_ac_third_term.reshape(batch_size, -1, 768)

    
     # ============== the corrected fourth term ================== xxx. for .788
    
    outputs_ac_fourth_term = torch.cat([outputs_ac_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1) # changed + 2 to + 1 to make it run
    outputs_ac_fourth_term = outputs_ac_fourth_term.reshape(batch_size, -1, 768)
    
    # ============== NOW CONCATENATE THEM =========
    
    
    ac_minus_representations = torch.cat([outputs_ac_first_term, outputs_ac_second_term, outputs_ac_third_term, outputs_ac_fourth_term], dim=-1)   
    
    
    
    ### ac minus span representation according to kuribayashi paper is now above.
    
    
     ############# FOR Fts #################
    
    outputs_fts = outputs[:,am_spans,:] # am spans for checking
    outputs_fts = torch.cat([outputs_fts[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_fts = outputs_fts.reshape(batch_size, nr_adus * 2, -1)
    
    
    
    outputs_fts_minus_one = outputs[:,fts_spans_minus_one,:] 
    outputs_fts_minus_one = torch.cat([outputs_fts_minus_one[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_fts_minus_one = outputs_fts_minus_one.reshape(batch_size, nr_adus * 2, -1)
    
    outputs_fts_plus_one = outputs[:,fts_spans_plus_one,:]
    outputs_fts_plus_one = torch.cat([outputs_fts_plus_one[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_fts_plus_one = outputs_fts_plus_one.reshape(batch_size, nr_adus * 2, -1)
    
    
    
    ### Now that we have outputs_am i.e. outputs at am_span indices, now create the four Kuri forumlas for AMs
    
    # ============== (batch_size x nr_adus x 768) =================== 
    
    outputs_fts_first_term = torch.cat([outputs_fts[:,i+1,:] - outputs_fts_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1) # i + 1 here means j in kuri, i here means i in kuri
    outputs_fts_first_term = outputs_fts_first_term.reshape(batch_size, -1, 768)
    
     # ============== the corrected 2nd one ==================
    
    outputs_fts_second_term = torch.cat([outputs_fts[:,i,:] - outputs_fts_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_fts_second_term = outputs_fts_second_term.reshape(batch_size, -1, 768)
    
        # ============== the corrected third one ================== 
    
    outputs_fts_third_term = torch.cat([outputs_fts_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_fts_third_term = outputs_fts_third_term.reshape(batch_size, -1, 768)
    

        # ============== the corrected fourth one ==================
    
    outputs_fts_fourth_term = torch.cat([outputs_fts_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_fts_fourth_term = outputs_fts_fourth_term.reshape(batch_size, -1, 768)
    
    # ============== NOW CONCATENATE THEM =========
    
    
    fts_minus_representations = torch.cat([outputs_fts_first_term, outputs_fts_second_term, outputs_fts_third_term, outputs_fts_fourth_term], dim=-1)   
    
    
    
    ### fts minus span representation according to kuribayashi paper is now above.    

    
    return am_minus_representations, ac_minus_representations, fts_minus_representations                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

## custom BERT model

In [22]:
class CustomBERTKuri(nn.Module):

    def __init__(self, first_model, model_am, model_ac, model_fts, nr_classes):
        
        super(CustomBERTKuri, self).__init__()
        
        self.first_model = first_model
        
        self.intermediate_linear_am = nn.Linear(3072, 768)
        self.intermediate_linear_ac = nn.Linear(3072, 768) 
        self.intermediate_linear_fts = nn.Linear(3072, 768)
        
        self.model_am = model_am
        self.model_ac = model_ac
        self.model_fts = model_fts
        
        self.nr_classes = nr_classes
                
        self.fc = nn.Linear(self.model_am.config.hidden_size + self.model_ac.config.hidden_size + self.model_fts.config.hidden_size, self.nr_classes)

    def forward(self, inputs):
        
        batch_tokenized, batch_spans = inputs 
        outputs = self.first_model(batch_tokenized)[0]
        am_minus_representations, ac_minus_representations, fts_minus_representations = get_span_representations(outputs, batch_spans)


        am_minus_representations = self.intermediate_linear_am(am_minus_representations)
        ac_minus_representations = self.intermediate_linear_ac(ac_minus_representations)
        fts_minus_representations = self.intermediate_linear_fts(fts_minus_representations)


        output_model_am = self.model_am(inputs_embeds = am_minus_representations)[0]
        output_model_ac = self.model_ac(inputs_embeds = ac_minus_representations)[0]
        output_model_fts = self.model_fts(inputs_embeds = fts_minus_representations)[0]

        adu_representations = torch.cat([output_model_am, output_model_ac, output_model_fts], dim=-1)
        output = self.fc(adu_representations)

        return output

## Run

In [23]:
NB_EPOCHS = 40
BATCH_SIZE = 28

In [24]:
first_model = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [25]:
model_am = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [26]:
model_ac = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [27]:
model_fts = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [28]:
custom_model = CustomBERTKuri(first_model, model_am, model_ac, model_fts, 3)

In [29]:
custom_model.to(device)

CustomBERTKuri(
  (first_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [30]:
loss = nn.CrossEntropyLoss(ignore_index=- 100)

In [31]:
optimizer = torch.optim.AdamW(custom_model.parameters(), lr=0.00023101297000831605)

In [33]:
NR_BATCHES = len(dataset['train']) / BATCH_SIZE
num_training_steps = NB_EPOCHS * NR_BATCHES
num_warmup_steps = int(0.2 * num_training_steps)

In [34]:
def lr_lambda(current_step: int):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return max(
            0.0, float(num_training_steps - current_step) / float(max(1, num_training_steps - num_warmup_steps))
        )

In [35]:
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

### create dataloaders

In [36]:
train_dataloader = DataLoader(dataset['train'], batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(dataset['validation'], batch_size=BATCH_SIZE, shuffle=True)

In [37]:
def flatten_list(list_of_lists):
    return [x for sublist in list_of_lists for x in sublist]

In [38]:
def remove_dummy_labels(test_preds, test_labels):
    
    idxes = []
    test_labels_l = []
    for idx, val in enumerate(test_labels):
        if val != -100:
            idxes.append(idx)
            test_labels_l.append(val)
    
    test_preds_l = []
    for idx, val in enumerate(test_preds):
        for good_idx in idxes:
            if idx == good_idx:
                test_preds_l.append(val)
        
    return test_preds_l, test_labels_l

## training 

In [41]:
def train(model, loss=None, optimizer=None, train_dataloader=None, val_dataloader=None, nb_epochs=20):
    """Training loop"""

    min_f1 = -torch.inf
    train_losses = []
    val_losses = []

    # Iterrate over epochs
    for e in range(nb_epochs):

        # Training
        train_loss = 0.0

        for i, batch in enumerate(tqdm(train_dataloader)):            
            
            #print(i)
            # unpack batch             
            labels = batch['label'].to(device)
            spans = batch['spans'].to(device)
            input_ids = batch['input_ids'].to(device)
            
            inputs = input_ids, spans
            
            # Reset gradients to 0
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(inputs)
            
            # Compute training loss
            current_loss = loss(outputs.flatten(0,1), labels.flatten())
            train_loss += current_loss.detach().item()

            # Compute gradients
            current_loss.backward()

            # Update weights
            optimizer.step()            
            
            del batch
        
        scheduler.step()
            
        
        # Validation
        val_loss = 0.0
        
        # torch.cuda.empty_cache()
        
        # Put model in eval mode
        model.eval()
        
        preds_l = []
        labels_l = []
        
        for batch in tqdm(val_dataloader):            
            
            # unpack batch             
            labels = batch['label'].to(device)
            spans = batch['spans'].to(device)
            input_ids = batch['input_ids'].to(device)
            
            inputs = input_ids, spans
            
            # Forward Pass
            outputs = model(inputs)

            # Compute validation loss
            current_loss = loss(outputs.flatten(0,1), labels.flatten())
            val_loss += current_loss.detach().item()
            
            preds_for_f1 = torch.argmax(outputs, dim=2).flatten().tolist()
            labels_for_f1 = labels.flatten().tolist()
            
            preds_l.append(preds_for_f1)
            labels_l.append(labels_for_f1)
            
            del batch
        
        # Prints
        
        preds_l = flatten_list(preds_l)
        labels_l = flatten_list(labels_l)
        
        preds_l, labels_l = remove_dummy_labels(preds_l, labels_l)
        
        f1_score_epoch = f1_score(preds_l, labels_l, average='macro')        
        
        print(f"Epoch {e+1}/{nb_epochs} \
                \t Training Loss: {train_loss/len(train_dataloader):.3f} \
                \t Validation Loss: {val_loss/len(val_dataloader):.3f} \
                \t F1 score: {f1_score_epoch}")
        
        train_losses.append(train_loss/len(train_dataloader))
        val_losses.append(val_loss/len(val_dataloader))
        

        # Save model if val loss decreases
        if f1_score_epoch > min_f1:

            min_f1 = f1_score_epoch
            torch.save(model.first_model.state_dict(), 'first_model.pt')
            torch.save(model.model_am.state_dict(), 'model_am.pt')
            torch.save(model.model_ac.state_dict(), 'model_ac.pt')
            torch.save(model.model_fts.state_dict(), 'model_fts.pt')
            torch.save(model.state_dict(), 'best_model.pt')
            
    return train_losses, val_losses

In [42]:
train_losses, val_losses = train(custom_model, loss, optimizer, train_dataloader, val_dataloader, NB_EPOCHS)

100%|██████████| 10/10 [00:03<00:00,  2.92it/s]


Epoch 1/40                 	 Training Loss: 1.039                 	 Validation Loss: 0.950                 	 F1 score: 0.3370873782117356


100%|██████████| 10/10 [00:03<00:00,  2.95it/s]


Epoch 2/40                 	 Training Loss: 0.856                 	 Validation Loss: 0.714                 	 F1 score: 0.44121288321546875


100%|██████████| 10/10 [00:03<00:00,  2.96it/s]


...

Epoch 38/40                 	 Training Loss: 0.015                 	 Validation Loss: 1.388                 	 F1 score: 0.7098260120170555


100%|██████████| 10/10 [00:03<00:00,  2.93it/s]


Epoch 39/40                 	 Training Loss: 0.011                 	 Validation Loss: 1.511                 	 F1 score: 0.695169711503155


100%|██████████| 10/10 [00:03<00:00,  2.93it/s]

Epoch 40/40                 	 Training Loss: 0.005                 	 Validation Loss: 1.612                 	 F1 score: 0.7186023905751916


### Predictions

In [43]:
test_dataloader = DataLoader(dataset['test'], batch_size=BATCH_SIZE, shuffle=True)

In [44]:
first_model = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
first_model.load_state_dict(torch.load('first_model.pt'))

<All keys matched successfully>

In [45]:
model_am = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
model_am.load_state_dict(torch.load('model_am.pt'))

<All keys matched successfully>

In [46]:
model_ac = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
model_ac.load_state_dict(torch.load('model_ac.pt'))

<All keys matched successfully>

In [47]:
model_fts = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
model_fts.load_state_dict(torch.load('model_fts.pt'))

<All keys matched successfully>

In [48]:
# Load best model

custom_model_2 = CustomBERTKuri(first_model, model_am, model_ac, model_fts, 3)
custom_model_2.load_state_dict(torch.load('best_model.pt'))

custom_model_2.to(device).eval()

CustomBERTKuri(
  (first_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [49]:
def predict(model, test_dataloader=None):
    
    """Prediction loop"""

    preds_l = []
    labels_l = []
    
    model.eval()

    for batch in test_dataloader:            
            
        # unpack batch             
        labels = batch['label'].to(device).flatten().tolist()
        spans = batch['spans'].to(device)
        input_ids = batch['input_ids'].to(device)
        
        inputs = input_ids, spans

        # get output
        
        raw_preds = model(inputs).to('cpu')
        # print(raw_preds.shape)
        raw_preds = raw_preds.detach()#.numpy()

        # Compute argmax
        
        predictions = torch.argmax(raw_preds, dim=2).flatten().tolist()
        preds_l.append(predictions)
        labels_l.append(labels)        
        
        del batch
            
    return flatten_list(preds_l), flatten_list(labels_l)

In [50]:
test_preds, test_labels = predict(custom_model_2, test_dataloader)

In [51]:
print(classification_report(test_labels, test_preds, digits=3))

              precision    recall  f1-score   support

        -100      0.000     0.000     0.000      3033
           0      0.084     0.813     0.153       155
           1      0.121     0.521     0.196       303
           2      0.462     0.857     0.601       805

    accuracy                          0.227      4296
   macro avg      0.167     0.548     0.237      4296
weighted avg      0.098     0.227     0.132      4296



/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
test_preds_l, test_labels_l = remove_dummy_labels(test_preds, test_labels)

In [53]:
print(classification_report(test_labels_l, test_preds_l, digits=3))

              precision    recall  f1-score   support

           0      0.689     0.813     0.746       155
           1      0.552     0.521     0.537       303
           2      0.869     0.857     0.863       805

    accuracy                          0.771      1263
   macro avg      0.703     0.730     0.715      1263
weighted avg      0.771     0.771     0.770      1263

